In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from skimage import io
from torch import nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from PIL import Image
import numpy as np
from torch.autograd import Variable

In [ ]:
!pip install pafy

In [ ]:
!pip install youtube-dl

In [ ]:
!pip install imageio==2.4.1

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/New_Code/New model')

In [ ]:
import vidsumm

In [ ]:
Path="/content/drive/MyDrive/New_Code/New model/trained_model19_1.pt"
retr_model1 = vidsumm.load_model5(Path)

In [ ]:
Path="/content/drive/MyDrive/New_Code/New model/trained_model19_1_glove.pt"
retr_model2 = vidsumm.load_model5_glove(Path)

In [ ]:
Path="/content/drive/MyDrive/New_Code/New model/trained_model9_1_glove.pt"
retr_model3 = vidsumm.load_model4_glove(Path)

In [ ]:
# Define the transform 
train_transform = transforms.Compose([
        transforms.Resize((224,224)),             # takes PIL image as input and outputs PIL image
        transforms.ToTensor(),              # takes PIL image as input and outputs torch.tensor
        transforms.Normalize(mean=[0.4280, 0.4106, 0.3589],  # takes tensor and outputs tensor
                             std=[0.2737, 0.2631, 0.2601]),  # see next step for mean and std
    ])

valid_transform = transforms.Compose([ 
        transforms.Resize((224,224)),             
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4280, 0.4106, 0.3589],
                             std=[0.2737, 0.2631, 0.2601]), 
    ])

test_transform = transforms.Compose([
        transforms.Resize((224,224)),             
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4280, 0.4106, 0.3589],
                             std=[0.2737, 0.2631, 0.2601]), 
    ])

In [ ]:
w2vmodel = vidsumm.get_word2vec_function()

In [ ]:
glove = vidsumm.get_glove_function()

In [ ]:
videoURL="https://www.youtube.com/watch?v=1L1RdktmbF0"
query="lion running"

In [ ]:
from vidsumm.utils_func import preprocess_video
imagenames = preprocess_video(query,videoURL)

In [ ]:
def prep_input(imagenames, query):
    
    ip_image = []
    for i in range(len(imagenames)):
        image = io.imread(imagenames[i])
        image = test_transform(Image.fromarray(image))
        ip_image.append(image)
    
    query = "lion running"
    query = query.lower()
    query = ' '.join(word for word in query.split(' ') if word in w2vmodel.vocab)
    words = query.split()
    SEQ_LENGTH = 8
    num_features = 300
    BATCH_SIZE = 1
    qdata = np.zeros((SEQ_LENGTH, num_features), dtype=np.float32)
    mask = np.ones((BATCH_SIZE, SEQ_LENGTH), dtype=np.bool)
    for j in range(SEQ_LENGTH):
        if j < len(words):
            qdata[j, :] = np.array(w2vmodel[str(words[j])])

    #qdata = qdata.mean(axis=0)

    qdata = torch.from_numpy(qdata)
    
    return ip_image, qdata

def prep_input_glove(imagenames, query):
    
    ip_image = []
    for i in range(len(imagenames)):
        image = io.imread(imagenames[i])
        image = test_transform(Image.fromarray(image))
        ip_image.append(image)
    
    query = "lion running"
    query = query.lower()
    query = ' '.join(word for word in query.split(' ') if word in glove.vocab)
    words = query.split()
    SEQ_LENGTH = 8
    num_features = 200
    BATCH_SIZE = 1
    qdata = np.zeros((SEQ_LENGTH, num_features), dtype=np.float32)
    mask = np.ones((BATCH_SIZE, SEQ_LENGTH), dtype=np.bool)
    for j in range(SEQ_LENGTH):
        if j < len(words):
            qdata[j, :] = np.array(glove[str(words[j])])

    #qdata = qdata.mean(axis=0)

    qdata = torch.from_numpy(qdata)
    
    return ip_image, qdata

In [ ]:
def predict1(ip_image, qdata):
    
    qdata = qdata.cuda()
    qdata = torch.unsqueeze(qdata,0)
    selected = {}
    for i in range(len(ip_image)):
        image = ip_image[i]
        image = image.cuda()
        image = torch.unsqueeze(image,0)

        with torch.no_grad():
            output = retr_model1(image, qdata)
        max_values_relevance, arg_maxs_relevance = torch.max(output, dim = 1)
        #print(arg_maxs_relevance)
        if(arg_maxs_relevance.item() >= 2):
            selected[i] = max_values_relevance.item()
        
    return selected

def predict2(ip_image, qdata):
    
    qdata = qdata.cuda()
    qdata = torch.unsqueeze(qdata,0)
    selected = {}
    for i in range(len(ip_image)):
        image = ip_image[i]
        image = image.cuda()
        image = torch.unsqueeze(image,0)

        with torch.no_grad():
            output = retr_model2(image, qdata)
        max_values_relevance, arg_maxs_relevance = torch.max(output, dim = 1)
        #print(arg_maxs_relevance)
        if(arg_maxs_relevance.item() >= 2):
            selected[i] = max_values_relevance.item()
        
    return selected

def predict3(ip_image, qdata):
    
    qdata = qdata.cuda()
    qdata = torch.unsqueeze(qdata,0)
    selected = {}
    for i in range(len(ip_image)):
        image = ip_image[i]
        image = image.cuda()
        image = torch.unsqueeze(image,0)

        with torch.no_grad():
            output = retr_model3(image, qdata)
        max_values_relevance, arg_maxs_relevance = torch.max(output, dim = 1)
        #print(arg_maxs_relevance)
        if(arg_maxs_relevance.item() >= 2):
            selected[i] = max_values_relevance.item()
        
    return selected

In [ ]:
ip_image, qdata = prep_input(imagenames, query)
ip_image_glove, qdata1_glove = prep_input_glove(imagenames, query)

In [ ]:
selected1 = predict1(ip_image, qdata)
selected2 = predict2(ip_image_glove, qdata1_glove)
selected3 = predict3(ip_image_glove, qdata1_glove)

For Model1

In [ ]:
from operator import itemgetter
selected = sorted(selected1.items(), key = itemgetter(1), reverse = True)[:10]
selected = sorted(selected, key = itemgetter(0))
selected

In [ ]:
for i in range(len(selected)):
    print(selected[i][0])

In [ ]:
import matplotlib .pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
plt.figure(figsize=(60, 10))
for i in range(len(selected)):
    plt.subplot(len(selected)/5,5, i+1);plt.imshow(mpimg.imread("/content/drive/MyDrive/New_Code/New model/videos/" + query + "/frames/"+str(selected[i][0])+".png"));plt.axis('off')
plt.show()

For Model 2

In [ ]:
selected = sorted(selected2.items(), key = itemgetter(1), reverse = True)[:10]
selected = sorted(selected, key = itemgetter(0))
selected

In [ ]:
for i in range(len(selected)):
    print(selected[i][0])

In [ ]:
import matplotlib .pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
plt.figure(figsize=(60, 10))
for i in range(len(selected)):
    plt.subplot(len(selected)/5,5, i+1);plt.imshow(mpimg.imread("/content/drive/MyDrive/New_Code/New model/videos/" + query + "/frames/"+str(selected[i][0])+".png"));plt.axis('off')
plt.show()

For Model 3

In [ ]:
selected = sorted(selected3.items(), key = itemgetter(1), reverse = True)[:10]
selected = sorted(selected, key = itemgetter(0))
selected

In [ ]:
for i in range(len(selected)):
    print(selected[i][0])

In [ ]:
import matplotlib .pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
plt.figure(figsize=(60, 10))
for i in range(len(selected)):
    plt.subplot(len(selected)/5,5, i+1);plt.imshow(mpimg.imread("/content/drive/MyDrive/New_Code/New model/videos/" + query + "/frames/"+str(selected[i][0])+".png"));plt.axis('off')
plt.show()